In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [3]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 54.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=8b35a273f5c1c76de0553cb59c80239e5414676c199913ad71fd9a52b2c8fb81
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-25 05:37:22--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.98MB/s    in 0.2s    

2022-04-25 05:37:23 (5.98 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [7]:
df

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [8]:
# Filter furniture dataframe where total votes are greater than or equal to 20
total_votes_df = df.filter("total_votes>=20")
total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [9]:
helpful_votes_df = total_votes_df.filter("helpful_votes/total_votes>=.50")
helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [10]:
print(helpful_votes_df.count())

18155


In [11]:
vine_review_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "Y")
vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35119071|R2BQOD1R0228FN|B00H2RSA88|     405483618|Sleep Innovations...|       Furniture|          3|           17|         26|   Y|                N|An okay product. ...|Three-stars is co...| 2015-08-27|
|         US|   44737123| RC31RUPFOHBHQ|B0125QZ50G|     350975212|Zinus Viscolatex ...|       Furniture|          4|    

In [12]:
isent_vine_review_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "N")
isent_vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [13]:
from pyspark.sql.functions import count

In [14]:
# Total number of reviews 
vine_review_count = vine_review_df.count()
print("Total Number of Reviews PAID and 'helpful': %f" % vine_review_count)

isent_vine_review_count = isent_vine_review_df.count()
print("Total Number of Reviews NON-PAID and 'helpful': %f" % isent_vine_review_count)

Total Number of Reviews PAID and 'helpful': 136.000000
Total Number of Reviews NON-PAID and 'helpful': 18019.000000


In [15]:
# Total number of ALL reviews
total_reviews_count = df.count()
print("Total Number of Furniture Reviews: %f" % total_reviews_count)

help_reviews_count = helpful_votes_df.count()
print("Total Number of 'Helpful' Furniture Reviews: %f" % help_reviews_count)

Total Number of Furniture Reviews: 792113.000000
Total Number of 'Helpful' Furniture Reviews: 18155.000000


In [16]:
# Number of 5-star Reviews 
fivestar_vine_df = vine_review_df.filter(isent_vine_review_df["star_rating"]=="5")
print("Total Number of 5-star Reviews PAID and 'helpful': %f" % fivestar_vine_df.count())


fivestar_isent_vine_df = isent_vine_review_df.filter(isent_vine_review_df["star_rating"]=="5")
print("Total Number of 5-star Reviews NON-PAID and 'helpful': %f" % fivestar_isent_vine_df.count())

Total Number of 5-star Reviews PAID and 'helpful': 74.000000
Total Number of 5-star Reviews NON-PAID and 'helpful': 8482.000000


In [17]:
#Total number of ALL 5-star reviews
fivestar_df = df.filter(df["star_rating"] == '5')
fivestar_df.show()

fivestar_count = fivestar_df.count()
print("Total Number of 5-Star Furniture Reviews: %f" % fivestar_count)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|            0|          0|   N|                Y|          Five Stars|          Great item| 2015-08-31|
|         US|    1272331|R1AIMEEPYHMOE4|B0030MPBZ4|     124663823|Bathroom Vanity T...|       Furniture|          5|    

In [18]:
# Percentage of 5-star Reviews
fivestar_vine_pct = (fivestar_vine_df.count()/vine_review_count)
print("Percent of 5-Star Furniture Reviews from PAID, 'helpful' dataset: %f" % fivestar_vine_pct)

fivestar_isent_vine_pct = (fivestar_isent_vine_df.count()/isent_vine_review_count)
print("Percent of 5-Star Furniture Reviews from NON-PAID, 'helpful' dataset: %f" % fivestar_isent_vine_pct)

Percent of 5-Star Furniture Reviews from PAID, 'helpful' dataset: 0.544118
Percent of 5-Star Furniture Reviews from NON-PAID, 'helpful' dataset: 0.470725


In [19]:
# Percentage of ALL 5-star reviews 

vine_fivestar_all_df = fivestar_df.filter(fivestar_df["vine"] == "Y")
vine_fivestar_all_count = vine_fivestar_all_df.count()

vine_fivestar_all_pct = (vine_fivestar_all_count/fivestar_count)
print("Number of 5-Star Furniture Reviews PAID: %f" % vine_fivestar_all_count)
print("Percent of 5-Star Furniture Reviews PAID: %f" % vine_fivestar_all_pct)


isent_vine_fivestar_all_df = fivestar_df.filter(fivestar_df["vine"] == "N")
isent_vine_fivestar_all_count = isent_vine_fivestar_all_df.count()

isent_vine_fivestar_all_pct = isent_vine_fivestar_all_count/fivestar_count
print("Number of 5-Star Furniture Reviews NON-PAID: %f" % isent_vine_fivestar_all_count)
print("Percent of 5-Star Furniture Reviews NON-PAID: %f" % isent_vine_fivestar_all_pct)

Number of 5-Star Furniture Reviews PAID: 1356.000000
Percent of 5-Star Furniture Reviews PAID: 0.003029
Number of 5-Star Furniture Reviews NON-PAID: 446360.000000
Percent of 5-Star Furniture Reviews NON-PAID: 0.996971


In [20]:
#Total number of 5-star HELPFUL reviews
fivestar_help_df = helpful_votes_df.filter(helpful_votes_df["star_rating"] == '5')
fivestar_help_df.show()

fivestar_help_count = fivestar_help_df.count()
print("Total Number of 5-Star 'Helpful' Furniture Reviews: %f" % fivestar_help_count)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [21]:
# Percentage of HELPFUL 5-star reviews 
 
vine_fivestar_help_df = fivestar_help_df.filter(fivestar_help_df["vine"] == "Y")
vine_fivestar_help_count = vine_fivestar_help_df.count()

vine_fivestar_help_pct = (vine_fivestar_help_count/fivestar_help_count)
print("Number of 5-Star, 'Helpful' Furniture Reviews PAID: %f" % vine_fivestar_help_count)
print("Percent of 5-Star,'Helpful'Furniture Reviews PAID: %f" % vine_fivestar_help_pct)
 
isent_vine_fivestar_help_df = fivestar_help_df.filter(fivestar_help_df["vine"] == "N")
isent_vine_fivestar_help_count = isent_vine_fivestar_help_df.count()

isent_vine_fivestar_help_pct = isent_vine_fivestar_help_count/fivestar_help_count
print("Number of 5-Star, 'Helpful' Furniture Reviews NON-PAID: %f" % isent_vine_fivestar_help_count)
print("Percent of 5-Star,'Helpful'Furniture Reviews NON-PAID: %f" % isent_vine_fivestar_help_pct)

Number of 5-Star, 'Helpful' Furniture Reviews PAID: 74.000000
Percent of 5-Star,'Helpful'Furniture Reviews PAID: 0.008649
Number of 5-Star, 'Helpful' Furniture Reviews NON-PAID: 8482.000000
Percent of 5-Star,'Helpful'Furniture Reviews NON-PAID: 0.991351
